In [1]:
fertilizer_json = {
    "Kalkammonsalpeter": {
        "Zusammensetzung": 0.27,
        "Emissionsfaktor": 7.101
    },
    "Harnstoff": {
        "Zusammensetzung": 0.46,
        "Emissionsfaktor": 12.4
    },
    "ASS": {
        "Zusammensetzung": 0.26,
        "Emissionsfaktor": 11.438
    },
    "Diammonphosphat": {
        "Zusammensetzung": {
            "N": 0.18,
            "P2O5": 0.46
        },
        "Emissionsfaktor": 11.958
    },
    "Gülle (Organisch)": {
        "Zusammensetzung": 1.0,
        "Emissionsfaktor": 4.97
    }
}

In [3]:
pip install geopandas

  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/c4/64/7d344cfcef5efddf9cf32f59af7f855828e9d74b5f862eddf5bfd9f25323/geopandas-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyogrio>=0.7.2 from https://files.pythonhosted.org/packages/94/8d/24f21e6a93ca418231aee3bddade7a0766c89c523832f29e08a8860f83e6/pyogrio-0.10.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/26/0c/b084e8839a117eaad8cb4fbaa81bbb24c6f183de0ee95c6c4e2770ab6f09/pyproj-3.7.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for shapely>=2.0.0 from https://files.pythonhosted.org/packages/b1/5a/6a67d929c467a1973b6bb9f0b00159cc343b02bf9a8d26db1abd2f87aa23/shapely-2.0.6-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/323.6 kB ? eta -:--:--
   --------------------------- ------------ 225.3/323.6 kB 4.7 MB/s eta 0:00:01
  

In [4]:
# Cell 1: Import Libraries
import geopandas as gpd
import pandas as pd
import glob
import os
from shapely.geometry import MultiPoint

# Set SHAPE_RESTORE_SHX environment variable
os.environ["SHAPE_RESTORE_SHX"] = "YES"

# Cell 2: Define Root Folder and Initialize List
root_folder = "data/Erntejahr 2022 (Weizen, Düngung)"

# Initialize an empty list to store shapefile paths
shapefile_list = []

# Traverse the directory structure
for subdir, _, _ in os.walk(root_folder):
    # Look for .shp files in the 'doc' folder of each subdirectory
    shapefiles = glob.glob(os.path.join(subdir, "doc", "*.shp"))
    shapefile_list.extend(shapefiles)


# Cell 3: Read Shapefiles and Combine into a Single GeoDataFrame
gdfs = []
for shapefile_path in shapefile_list:
    gdf = gpd.read_file(shapefile_path)
    # Append the GeoDataFrame to a list
    gdfs.append(gdf)

# Concatenate all GeoDataFrames into a single GeoDataFrame
combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# Fill missing values with NA
combined_gdf = combined_gdf.fillna('NA')

# Cell 4: Display Output and Columns
print("All shapefiles combined successfully!")
# Uncomment the lines below if you want to save the combined shapefile
# Cell 5: Save Combined GeoDataFrame as a Shapefile
# output_shapefile_path = os.path.join(root_folder, "combined_shapefile.shp")
# combined_gdf.to_file(output_shapefile_path)


ValueError: No objects to concatenate

In [27]:
combined_gdf['Product'] = combined_gdf['Product'].apply(lambda x: x.split()[0])

# Cell 6: Calculate Emissions for Each Product
def calculate_emissions(row):
    product = row["Product"]
    appliedRate = row["AppliedRate"]
    if product in fertilizer_json:
        if fertilizer_json[product]["Zusammensetzung"]:
            if isinstance(fertilizer_json[product]["Zusammensetzung"], dict):
                composition = fertilizer_json[product]["Zusammensetzung"]["N"]
            else:
                composition = fertilizer_json[product]["Zusammensetzung"]
            emissions_factor = fertilizer_json[product]["Emissionsfaktor"]
            emissions = composition * appliedRate * emissions_factor
        else:
            emissions = appliedRate * fertilizer_json[product]["Emissionsfaktor"]
    else:
        emissions = 0
    return emissions

combined_gdf["Emissions"] = combined_gdf.apply(calculate_emissions, axis=1)
combined_gdf

,Time,Heading,DISTANCE,SWATHWIDTH,Product,SECTIONID,AppliedRate,ControlRate,TargetRate,Elevation,IsoTime,Machine,FUEL,VEHICLSPEED,geometry,Emissions
0,3/14/2022 11:09:16 AM,37.606482,1.691935,7.500,Gärsubstrat,592,0.0,0.0,0.0,267.132457,2022-03-14T11:09:16.381Z,1,0.001947,6.679704,POINT (8.96777 49.23514),0.0000
1,3/14/2022 11:09:16 AM,37.606482,1.691935,7.500,Gärsubstrat,593,0.0,0.0,0.0,267.132457,2022-03-14T11:09:16.381Z,1,0.001947,6.679704,POINT (8.96785 49.23509),0.0000
2,3/14/2022 11:09:17 AM,47.731289,1.733935,7.500,Gärsubstrat,592,0.0,0.0,0.0,267.239457,2022-03-14T11:09:17.380Z,1,0.002859,6.784824,POINT (8.96779 49.23515),0.0000
3,3/14/2022 11:09:17 AM,47.731289,1.733935,7.500,Gärsubstrat,593,0.0,0.0,0.0,267.239457,2022-03-14T11:09:17.380Z,1,0.002859,6.784824,POINT (8.96786 49.2351),0.0000
4,3/14/2022 11:09:25 AM,50.685905,0.073935,7.500,Gärsubstrat,592,0.0,0.0,0.0,266.942457,2022-03-14T11:09:25.571Z,1,0.001361,0.188454,POINT (8.96774 49.23513),0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103070,3/1/2022 9:03:29 AM,139.796797,5.086113,1.874,ASS,2812,0.0,0.0,0.0,283.718461,2022-03-01T09:03:29.503Z,1,0.001915,18.513573,POINT (8.96912 49.23925),0.0000
103071,3/1/2022 9:03:43 AM,136.195835,3.887845,1.874,ASS,2812,60.0,0.0,60.0,289.067898,2022-03-01T09:03:43.503Z,1,0.002579,15.290286,POINT (8.96972 49.23876),178.4328
103072,3/1/2022 9:03:44 AM,127.000221,3.363845,1.874,ASS,2812,60.0,0.0,60.0,289.320898,2022-03-01T09:03:44.503Z,1,0.007307,13.457166,POINT (8.96975 49.23871),178.4328
103073,3/1/2022 9:04:25 AM,90.636547,1.691301,1.874,ASS,2798,0.0,0.0,0.0,289.629933,2022-03-01T09:04:25.503Z,1,0.001150,5.618586,POINT (8.97221 49.23942),0.0000


In [28]:
average_emissions = combined_gdf['Emissions'].mean()
print(average_emissions)

38.63634890322581


In [30]:
combined_gdf_grouped = combined_gdf.groupby("Product").agg({"Emissions": "sum"})
print(combined_gdf_grouped)

                     Emissions
Product                       
ASS               3.982442e+06
Gärsubstrat       0.000000e+00
Kalkamonsalpeter  0.000000e+00
